In [1]:

import json
from src.base_creature_builder import creature_template_builder
from src.validator import validate_creature, finalization
from src.weapons_builder import weapon_template_builder
from src.utils import fill_template_with_openai, generate_id, upload_new_character, get_character, put_character, get_bearer_token
from src.base_creature_builder import CREATURE_TEMPLATE_DICT
from src.validator import validate_weapon
    

In [2]:
final_creature_dict = CREATURE_TEMPLATE_DICT
bearer_token = get_bearer_token()

In [ ]:
# function still TBD
complete_builder(
    creature_description="A goblin that throws pies at its enemies, it keeps the pies in a satchel at its side.",
    creature_name="Pie Goblin",
    creature_type="Minion",
    setting_description="A generic D&D-like fantasty setting.",
    skills=["Stealth", "Ranged (Light)"],
    allow_other_skills=True,
    combat_cr=2,
    social_cr=1,
    agility=3,
    cunning=3,
    presence=1
)
# calls creature_template_builder, weapon_template_builder, and finalization

In [3]:
creature_description = "A goblin that throws pies at its enemies, it keeps the pies in a satchel at its side."
setting_description = "A generic D&D-like fantasty setting."
creature_type = "Minion"
creature_builder_string = creature_template_builder(
    creature_description=creature_description,
    creature_name="Pie Goblin",
    creature_type=creature_type,
    setting_description=setting_description,
    skills=["Stealth", "Ranged (Light)"],
    allow_other_skills=True,
    combat_cr=2,
    social_cr=1,
    agility=3,
    cunning=3,
    presence=1)
generated_character_data = fill_template_with_openai(creature_builder_string)
generated_character_data_as_dict = validate_creature(generated_character_data)
generated_character_data_as_dict['characterDescription'] = creature_description
final_creature_dict["characters"] = [generated_character_data_as_dict]

passes all validation steps


In [4]:

weapon_builder_string = weapon_template_builder(
    creature_name=generated_character_data_as_dict['name'],
    creature_description=creature_description, 
    creature_type=creature_type, 
    setting_description=setting_description, 
    creature_skills=str(list(generated_character_data_as_dict['masterSkills'].keys())), 
    combat_cr=generated_character_data_as_dict['combatCR'],
    number_of_weapons=3)

generated_weapon_data = fill_template_with_openai(weapon_builder_string)


In [5]:
generated_weapon_data_as_dict = validate_weapon(generated_weapon_data)
final_creature_dict["customWeapons"] = generated_weapon_data_as_dict

passes all validation steps
passes all validation steps
passes all validation steps


In [6]:
final_creature_dict["characters"][0]["equipmentWeapons"] = {}
for weapon in generated_weapon_data_as_dict:
    new_name = weapon['name'].replace(' ', '')  # Remove spaces
    new_name = new_name[0].lower() + new_name[1:]  # Lowercase the first letter
    final_creature_dict["characters"][0]["equipmentWeapons"][weapon['id']] = {"craftsmanship": "",
                    "carried": True,
                    "id": new_name,
                    "equipped": True}

In [7]:
final_creature_dict

{'customArchetypes': [{'name': 'NPC Archetype',
   'Brawn': 1,
   'Agility': 1,
   'Intellect': 1,
   'Cunning': 1,
   'Willpower': 1,
   'Presence': 1,
   'woundThreshold': 5,
   'strainThreshold': 1,
   'experience': '1',
   'description': '',
   'setting': ['All'],
   'skills': {},
   'talents': [],
   'id': 'ByO058vfgGB33EOrabeE'}],
 'customCareers': [{'name': 'NPC Career',
   'description': '',
   'setting': ['All'],
   'skills': [],
   'id': '9VWOCKS9chsyHcBjS1U3'}],
 'characters': [{'name': 'Pie Goblin',
   'careerSkillsRank': ['Stealth', 'RangedLight', 'Athletics', 'Deception'],
   'masterSkills': {'Stealth': {'careerRank': 2},
    'Deception': {'careerRank': 1},
    'RangedLight': {'careerRank': 2}},
   'creatureType': 'Minion',
   'adversaryLevel': 0,
   'minionCount': 3,
   'creationCharacteristics': {'Brawn': 1,
    'Agility': 2,
    'Intellect': 1,
    'Cunning': 2,
    'Willpower': 0,
    'Presence': 0},
   'combatCR': 2,
   'socialCR': 1,
   'generalCR': 2,
   'woundThre

In [8]:
#final_data = finalization(generated_character_data_as_dict, None)
with open("creature.json", "w") as f:
    f.write(json.dumps(final_creature_dict))
f.close()

In [9]:
import requests
session = requests.Session()
def upload_new_character(bearer_token, creature_json_path="creature.json"):
    url = "https://api.rpgsessions.com/import/emporium"
    headers = {
        "Authorization": "Bearer {bearer_token}".format(bearer_token=bearer_token),
    }

    # Open the JSON file in binary mode
    with open(creature_json_path, 'rb') as f:
        # Create a dictionary that will be sent as the files parameter
        files = {'file': (creature_json_path, f)}

        # Make the request
        response = session.post(url, headers=headers, files=files)

    print(response.json())
    character_id = response.json()[0]['id']
    return character_id

character_id = upload_new_character(bearer_token, creature_json_path="creature.json")

[{'name': 'Pie Goblin', 'id': '6491e35e18fddf5357135bf8'}]


In [11]:


data = get_character(character_id, bearer_token)
#print(data)

In [12]:
creature_type = generated_character_data_as_dict["creatureType"]
soak = generated_character_data_as_dict["soak"]
wounds_threshold = generated_character_data_as_dict["woundThreshold"]
strain_threshold = generated_character_data_as_dict["strainThreshold"]
defense_melee = generated_character_data_as_dict["defenseMelee"]
defense_ranged = generated_character_data_as_dict["defenseRanged"]
minion_count = generated_character_data_as_dict["minionCount"]-1
adversary_level = generated_character_data_as_dict["adversaryLevel"]
combat_cr = generated_character_data_as_dict["combatCR"]
social_cr = generated_character_data_as_dict["socialCR"]
general_cr = generated_character_data_as_dict["generalCR"]

In [13]:
data["characterType"] = "[character type] {creature_type}".format(
    creature_type=creature_type.lower()
)

data["configuration"] = {
    "diceTheme": "[dice theme] genesys",
    "gameTheme": "[game theme] genesys terrinoth",
    "ndsCharacterType": "[nds character type] {character_type}".format(
        character_type=creature_type.lower()
    ),
    "talentsDisplayType": "[nds character talent display type] list",
    "talentsSignatureAbilitiesEnabled": False,
    "forcePowersDisplayType": "[nds character talent display type] list group",
    "dutyEnabled": False,
    "forceEnabled": False,
    "forceDiceEnabled": False,
    "moralityEnabled": False,
    "obligationEnabled": False,
    "superCharacteristicsEnabled": False,
    "heroicAbilitiesEnabled": False,
    "aemberEnabled": False,
    "favorsEnabled": False,
    "agendasEnabled": False,
    "automations": False,
}
data["attributes"] = [
    {"value": soak, "type": "[nds character attribute] soak"},
    {"value": 0, "type": "[nds character attribute] wounds current"},
    {"value": wounds_threshold, "type": "[nds character attribute] wounds threshold"},
    {"value": 0, "type": "[nds character attribute] strain current"},
    {"value": strain_threshold, "type": "[nds character attribute] strain threshold"},
    {"value": defense_ranged, "type": "[nds character attribute] defense ranged"},
    {"value": defense_melee, "type": "[nds character attribute] defense melee"},
    {"value": 0, "type": "[nds character attribute] available xp"},
    {"value": 0, "type": "[nds character attribute] total xp"},
    {"value": 0, "type": "[nds character attribute] money"},
    {"value": 0, "type": "[nds character attribute] encumbrance current"},
    {"value": 5, "type": "[nds character attribute] encumbrance threshold"},
    {"value": 0, "type": "[nds character attribute] force rating committed"},
    {"value": minion_count, "type": "[nds character attribute] current minion count"},
    {"value": minion_count, "type": "[nds character attribute] character count"},
    {"value": adversary_level, "type": "[nds character attribute] adversary level"},
    {"value": combat_cr, "type": "[nds character attribute] combat power level"},
    {"value": social_cr, "type": "[nds character attribute] social power level"},
    {"value": general_cr, "type": "[nds character attribute] general power level"},
    {"value": 0, "type": "[nds character attribute] available aember"},
    {"value": 0, "type": "[nds character attribute] total aember"},
]

In [14]:
from src.utils import put_character
put_character(character_id, bearer_token, data)

<Response [200]>

In [ ]:
# TODO: 
# set what skills I want in the setting and then remove from the list those I don't want
# custom skills made
# abilities still need to be added
# (custom) gear and equipped equiment too


In [1]:
from src.main import complete_builder

complete_builder(
    creature_description="An undead unicorn.",
    creature_name="Undeadicorn",
    creature_type="Nemesis",
    setting_description="A generic fantasy setting with magic and monsters",
    setting_specific_skills=[],
    setting_removed_skils=[],
    skills=["Melee"],
    allow_other_skills=True,
    combat_cr=10,
    social_cr=1,
    general_cr=-1,
    brawn=-1,
    agility=-1,
    intellect=-1,
    cunning=-1,
    willpower=-1,
    presence=-1,
    number_of_weapons=3,
)

Getting bearer token
Building creature template
Filling character template with OpenAI, expect this to take about 30 seconds.
Validating character data
passes all creature validation steps
Building weapon template
Filling weapon template with OpenAI, expect this to take about 30 seconds.
Validating weapon data
passes all weapon validation steps
passes all weapon validation steps
passes all weapon validation steps
Adding weapons to character
Writing character to file
Uploading character to RPGSessions
[{'name': 'Undeadicorn', 'id': '6491ea554fbaea0966c7e708'}]
Getting character from RPGSessions to edit raw data
Editing character data
Re-uploading character data to RPGSessions, overwriting existing character we just made
Done!
